In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import multivariate_normal as mvn

from LSTM import LSTM

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

# set plot parameters
params = {'legend.fontsize': 18,
          'figure.figsize': (16, 12),
          'lines.linewidth': 4,
          'axes.labelsize': 24,
          'axes.titlesize':24,
          'axes.linewidth':5,
          'xtick.labelsize':20,
          'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-rwys47u7 because the default path (/home/jaron/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Set parameters of script

In [2]:
# define k for k-fold (n_splits = k)
n_splits = 20

# number of times to run k-fold x-validation 
# each trial uses different random partitioning of data
n_trials = 10

# import data

In [3]:
df = pd.read_csv("Data/2021_02_19_MultifunctionalDynamicData.csv")

all_experiments = df.Experiments.values
unique_experiments = np.unique(all_experiments)

df.head()

,Experiments,Time,PC_OD,PJ_OD,BV_OD,BF_OD,BO_OD,BT_OD,BC_OD,BY_OD,...,CG_OD,ER_OD,RI_OD,CC_OD,DL_OD,DF_OD,Butyrate,Acetate,Lactate,Succinate
0,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,0,0.000471,0.0,0.000471,0.000000,0.000000,0.000471,0.000471,0.0,...,0.000471,0.000471,0.000471,0.0,0.0,0.000471,0.000000,0.000000,28.000000,0.000000
1,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,16,0.465116,0.0,0.029207,0.000000,0.000000,0.249717,0.500651,0.0,...,0.024339,0.327601,0.001460,0.0,0.0,0.392830,23.092697,47.849302,18.910852,26.141885
2,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,32,0.104523,0.0,0.027928,0.000000,0.000000,0.220107,0.380210,0.0,...,0.020739,0.293384,0.000830,0.0,0.0,0.280111,23.996267,38.915218,17.977137,26.884748
3,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,48,0.124852,0.0,0.012194,0.000000,0.000000,0.268268,0.333970,0.0,...,0.023660,0.188188,0.002366,0.0,0.0,0.224952,24.839219,34.325914,19.406971,31.628061
4,BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-E...,0,0.000000,0.0,0.000388,0.000388,0.000388,0.000388,0.000000,0.0,...,0.000388,0.000388,0.000388,0.0,0.0,0.000000,0.000000,0.000000,28.000000,0.000000


### Define which variables are species and which are metabolites

In [4]:
# specify species and metabolite names 
species = df.columns.values[2:-4]
metabolites =  df.columns.values[-4:]
controls = []
system_variables = np.concatenate((np.array(species), np.array(metabolites)))
system_variables

array(['PC_OD', 'PJ_OD', 'BV_OD', 'BF_OD', 'BO_OD', 'BT_OD', 'BC_OD',
       'BY_OD', 'BU_OD', 'DP_OD', 'BL_OD', 'BA_OD', 'BP_OD', 'CA_OD',
       'EL_OD', 'FP_OD', 'CH_OD', 'AC_OD', 'BH_OD', 'CG_OD', 'ER_OD',
       'RI_OD', 'CC_OD', 'DL_OD', 'DF_OD', 'Butyrate', 'Acetate',
       'Lactate', 'Succinate'], dtype=object)

In [5]:
# perform k-fold x-validation several times (n_trials) with randomized partitions of data
for k in range(n_trials):
    
    # initialize dataframe to save predictions
    df_kfold = pd.DataFrame()

    # object to get indices of kfold splits
    kf = KFold(n_splits=n_splits, random_state=k, shuffle=True)

    # loop over k-fold splits
    for train_index, test_index in kf.split(unique_experiments):

        # update index values to pull trajectories 
        train_index = np.in1d(all_experiments, unique_experiments[train_index])
        test_index = np.in1d(all_experiments, unique_experiments[test_index])

        # pull train and test indices
        train_df  = df.iloc[train_index].copy() 
        test_df   = df.iloc[test_index].copy() 

        # train LSTM 
        lstm = LSTM(train_df, system_variables, hidden_size=4098)
        lstm.train(train_df)

        # Make predictions
        pred_df = lstm.predict(test_df)

        # save data
        df_kfold = pd.concat((df_kfold, pred_df))
        
    # save kfold predictions
    df_kfold.to_csv(f"Fig2/Kfold/LSTM_{n_splits}_fold_{k+1}.csv", index=False)

Epoch: 1/200, Train Loss: 0.72241
Epoch: 2/200, Train Loss: 0.51605
Epoch: 3/200, Train Loss: 0.38492
Epoch: 4/200, Train Loss: 0.33010
Epoch: 5/200, Train Loss: 0.26059
Epoch: 6/200, Train Loss: 0.24767
Epoch: 7/200, Train Loss: 0.23179
Epoch: 8/200, Train Loss: 0.20286
Epoch: 9/200, Train Loss: 0.18636
Epoch: 10/200, Train Loss: 0.19072
Epoch: 11/200, Train Loss: 0.19258
Epoch: 12/200, Train Loss: 0.14941
Epoch: 13/200, Train Loss: 0.14815
Epoch: 14/200, Train Loss: 0.12186
Epoch: 15/200, Train Loss: 0.11046
Epoch: 16/200, Train Loss: 0.10443
Epoch: 17/200, Train Loss: 0.11386
Epoch: 18/200, Train Loss: 0.10623
Epoch: 19/200, Train Loss: 0.10569
Epoch: 20/200, Train Loss: 0.10430
Epoch: 21/200, Train Loss: 0.10498
Epoch: 22/200, Train Loss: 0.10292
Epoch: 23/200, Train Loss: 0.10020
Epoch: 24/200, Train Loss: 0.10021
Epoch: 25/200, Train Loss: 0.09831
Epoch: 26/200, Train Loss: 0.09387
Epoch: 27/200, Train Loss: 0.09823
Epoch: 28/200, Train Loss: 0.09708
Epoch: 29/200, Train Loss: 0.

Epoch: 33/200, Train Loss: 0.09307
Epoch: 34/200, Train Loss: 0.09794
Epoch: 35/200, Train Loss: 0.09401
Epoch: 36/200, Train Loss: 0.09967
Epoch: 37/200, Train Loss: 0.09859
Epoch: 38/200, Train Loss: 0.08814
Epoch: 39/200, Train Loss: 0.09398
Epoch: 40/200, Train Loss: 0.09321
Epoch: 41/200, Train Loss: 0.09807
Epoch: 42/200, Train Loss: 0.09076
Epoch: 43/200, Train Loss: 0.09017
Epoch: 44/200, Train Loss: 0.08867
Epoch: 45/200, Train Loss: 0.09012
Epoch: 46/200, Train Loss: 0.09595
Epoch: 47/200, Train Loss: 0.09074
Epoch: 48/200, Train Loss: 0.09448
Epoch: 49/200, Train Loss: 0.08684
Epoch: 50/200, Train Loss: 0.09442
Epoch: 51/200, Train Loss: 0.09641
Epoch: 52/200, Train Loss: 0.09004
Epoch: 53/200, Train Loss: 0.09050
Epoch: 54/200, Train Loss: 0.09004
Epoch: 55/200, Train Loss: 0.09171
Epoch: 56/200, Train Loss: 0.08884
Epoch: 57/200, Train Loss: 0.09411
Epoch: 58/200, Train Loss: 0.08947
Epoch: 59/200, Train Loss: 0.08434
Epoch: 60/200, Train Loss: 0.09154
Epoch: 61/200, Train

Epoch: 65/200, Train Loss: 0.09331
Epoch: 66/200, Train Loss: 0.09067
Epoch: 67/200, Train Loss: 0.09254
Epoch: 68/200, Train Loss: 0.09668
Epoch: 69/200, Train Loss: 0.09015
Epoch: 70/200, Train Loss: 0.09388
Epoch: 71/200, Train Loss: 0.08951
Epoch: 72/200, Train Loss: 0.08903
Epoch: 73/200, Train Loss: 0.09182
Epoch: 74/200, Train Loss: 0.09394
Epoch: 75/200, Train Loss: 0.09428
Epoch: 76/200, Train Loss: 0.09073
Epoch: 77/200, Train Loss: 0.09175
Epoch: 78/200, Train Loss: 0.09434
Epoch: 79/200, Train Loss: 0.08952
Epoch: 80/200, Train Loss: 0.09545
Epoch: 81/200, Train Loss: 0.08767
Epoch: 82/200, Train Loss: 0.08984
Epoch: 83/200, Train Loss: 0.09551
Epoch: 84/200, Train Loss: 0.09278
Epoch: 85/200, Train Loss: 0.09445
Epoch: 86/200, Train Loss: 0.09388
Epoch: 87/200, Train Loss: 0.09307
Epoch: 88/200, Train Loss: 0.08691
Epoch: 89/200, Train Loss: 0.09078
Epoch: 90/200, Train Loss: 0.08964
Epoch: 91/200, Train Loss: 0.09199
Epoch: 92/200, Train Loss: 0.09007
Epoch: 93/200, Train

Epoch: 97/200, Train Loss: 0.10093
Epoch: 98/200, Train Loss: 0.09608
Epoch: 99/200, Train Loss: 0.09569
Epoch: 100/200, Train Loss: 0.09784
Epoch: 101/200, Train Loss: 0.09423
Epoch: 102/200, Train Loss: 0.09084
Epoch: 103/200, Train Loss: 0.09920
Epoch: 104/200, Train Loss: 0.10021
Epoch: 105/200, Train Loss: 0.09133
Epoch: 106/200, Train Loss: 0.09276
Epoch: 107/200, Train Loss: 0.09433
Epoch: 108/200, Train Loss: 0.09890
Epoch: 109/200, Train Loss: 0.09032
Epoch: 110/200, Train Loss: 0.09078
Epoch: 111/200, Train Loss: 0.09477
Epoch: 112/200, Train Loss: 0.09269
Epoch: 113/200, Train Loss: 0.09949
Epoch: 114/200, Train Loss: 0.09818
Epoch: 115/200, Train Loss: 0.09219
Epoch: 116/200, Train Loss: 0.09242
Epoch: 117/200, Train Loss: 0.09289
Epoch: 118/200, Train Loss: 0.09153
Epoch: 119/200, Train Loss: 0.09489
Epoch: 120/200, Train Loss: 0.09828
Epoch: 121/200, Train Loss: 0.09291
Epoch: 122/200, Train Loss: 0.09247
Epoch: 123/200, Train Loss: 0.09645
Epoch: 124/200, Train Loss: 0.0

Epoch: 128/200, Train Loss: 0.09445
Epoch: 129/200, Train Loss: 0.09057
Epoch: 130/200, Train Loss: 0.09455
Epoch: 131/200, Train Loss: 0.09976
Epoch: 132/200, Train Loss: 0.09593
Epoch: 133/200, Train Loss: 0.10073
Epoch: 134/200, Train Loss: 0.09037
Epoch: 135/200, Train Loss: 0.09772
Epoch: 136/200, Train Loss: 0.09278
Epoch: 137/200, Train Loss: 0.09708
Epoch: 138/200, Train Loss: 0.09805
Epoch: 139/200, Train Loss: 0.09388
Epoch: 140/200, Train Loss: 0.08736
Epoch: 141/200, Train Loss: 0.09351
Epoch: 142/200, Train Loss: 0.09562
Epoch: 143/200, Train Loss: 0.09476
Epoch: 144/200, Train Loss: 0.09226
Epoch: 145/200, Train Loss: 0.08939
Epoch: 146/200, Train Loss: 0.09385
Epoch: 147/200, Train Loss: 0.09719
Epoch: 148/200, Train Loss: 0.08902
Epoch: 149/200, Train Loss: 0.09109
Epoch: 150/200, Train Loss: 0.08903
Epoch: 151/200, Train Loss: 0.09237
Epoch: 152/200, Train Loss: 0.09495
Epoch: 153/200, Train Loss: 0.09404
Epoch: 154/200, Train Loss: 0.08806
Epoch: 155/200, Train Loss: 

...

Epoch: 106/200, Train Loss: 0.09372
Epoch: 107/200, Train Loss: 0.09310
Epoch: 108/200, Train Loss: 0.09840
Epoch: 109/200, Train Loss: 0.08987
Epoch: 110/200, Train Loss: 0.09890
Epoch: 111/200, Train Loss: 0.09657
Epoch: 112/200, Train Loss: 0.08647
Epoch: 113/200, Train Loss: 0.09166
Epoch: 114/200, Train Loss: 0.09615
Epoch: 115/200, Train Loss: 0.09177
Epoch: 116/200, Train Loss: 0.09088
Epoch: 117/200, Train Loss: 0.10058
Epoch: 118/200, Train Loss: 0.09085
Epoch: 119/200, Train Loss: 0.09536
Epoch: 120/200, Train Loss: 0.09221
Epoch: 121/200, Train Loss: 0.09625
Epoch: 122/200, Train Loss: 0.09435
Epoch: 123/200, Train Loss: 0.08830
Epoch: 124/200, Train Loss: 0.09520
Epoch: 125/200, Train Loss: 0.09772
Epoch: 126/200, Train Loss: 0.09393
Epoch: 127/200, Train Loss: 0.09308
Epoch: 128/200, Train Loss: 0.09307
Epoch: 129/200, Train Loss: 0.09432
Epoch: 130/200, Train Loss: 0.09309
Epoch: 131/200, Train Loss: 0.09674
Epoch: 132/200, Train Loss: 0.09129
Epoch: 133/200, Train Loss: 

Epoch: 137/200, Train Loss: 0.11105
Epoch: 138/200, Train Loss: 0.10541
Epoch: 139/200, Train Loss: 0.10795
Epoch: 140/200, Train Loss: 0.10874
Epoch: 141/200, Train Loss: 0.10898
Epoch: 142/200, Train Loss: 0.11049
Epoch: 143/200, Train Loss: 0.11516
Epoch: 144/200, Train Loss: 0.10170
Epoch: 145/200, Train Loss: 0.10417
Epoch: 146/200, Train Loss: 0.10841
Epoch: 147/200, Train Loss: 0.10726
Epoch: 148/200, Train Loss: 0.10931
Epoch: 149/200, Train Loss: 0.10617
Epoch: 150/200, Train Loss: 0.11000
Epoch: 151/200, Train Loss: 0.10654
Epoch: 152/200, Train Loss: 0.10699
Epoch: 153/200, Train Loss: 0.10956
Epoch: 154/200, Train Loss: 0.10355
Epoch: 155/200, Train Loss: 0.11398
Epoch: 156/200, Train Loss: 0.11257
Epoch: 157/200, Train Loss: 0.10984
Epoch: 158/200, Train Loss: 0.11373
Epoch: 159/200, Train Loss: 0.10520
Epoch: 160/200, Train Loss: 0.10811
Epoch: 161/200, Train Loss: 0.10953
Epoch: 162/200, Train Loss: 0.10959
Epoch: 163/200, Train Loss: 0.11030
Epoch: 164/200, Train Loss: 

Epoch: 168/200, Train Loss: 0.13259
Epoch: 169/200, Train Loss: 0.13118
Epoch: 170/200, Train Loss: 0.13415
Epoch: 171/200, Train Loss: 0.13620
Epoch: 172/200, Train Loss: 0.13849
Epoch: 173/200, Train Loss: 0.13615
Epoch: 174/200, Train Loss: 0.13447
Epoch: 175/200, Train Loss: 0.13159
Epoch: 176/200, Train Loss: 0.13503
Epoch: 177/200, Train Loss: 0.13489
Epoch: 178/200, Train Loss: 0.13127
Epoch: 179/200, Train Loss: 0.13079
Epoch: 180/200, Train Loss: 0.13716
Epoch: 181/200, Train Loss: 0.13590
Epoch: 182/200, Train Loss: 0.13762
Epoch: 183/200, Train Loss: 0.13115
Epoch: 184/200, Train Loss: 0.13438
Epoch: 185/200, Train Loss: 0.13237
Epoch: 186/200, Train Loss: 0.13293
Epoch: 187/200, Train Loss: 0.13922
Epoch: 188/200, Train Loss: 0.12819
Epoch: 189/200, Train Loss: 0.13541
Epoch: 190/200, Train Loss: 0.13692
Epoch: 191/200, Train Loss: 0.13188
Epoch: 192/200, Train Loss: 0.13228
Epoch: 193/200, Train Loss: 0.13410
Epoch: 194/200, Train Loss: 0.13431
Epoch: 195/200, Train Loss: 

Epoch: 199/200, Train Loss: 0.10765
Epoch: 200/200, Train Loss: 0.10821
Epoch: 1/200, Train Loss: 0.78291
Epoch: 2/200, Train Loss: 0.58243
Epoch: 3/200, Train Loss: 0.49940
Epoch: 4/200, Train Loss: 0.60173
Epoch: 5/200, Train Loss: 0.58444
Epoch: 6/200, Train Loss: 0.50812
Epoch: 7/200, Train Loss: 0.48168
Epoch: 8/200, Train Loss: 0.33773
Epoch: 9/200, Train Loss: 0.33225
Epoch: 10/200, Train Loss: 0.30661
Epoch: 11/200, Train Loss: 0.31205
Epoch: 12/200, Train Loss: 0.28154
Epoch: 13/200, Train Loss: 0.21173
Epoch: 14/200, Train Loss: 0.18285
Epoch: 15/200, Train Loss: 0.16203
Epoch: 16/200, Train Loss: 0.15386
Epoch: 17/200, Train Loss: 0.14838
Epoch: 18/200, Train Loss: 0.14202
Epoch: 19/200, Train Loss: 0.13881
Epoch: 20/200, Train Loss: 0.14252
Epoch: 21/200, Train Loss: 0.14029
Epoch: 22/200, Train Loss: 0.13704
Epoch: 23/200, Train Loss: 0.13949
Epoch: 24/200, Train Loss: 0.12390
Epoch: 25/200, Train Loss: 0.12099
Epoch: 26/200, Train Loss: 0.12190
Epoch: 27/200, Train Loss: 

Epoch: 31/200, Train Loss: 0.12625
Epoch: 32/200, Train Loss: 0.12831
Epoch: 33/200, Train Loss: 0.12482
Epoch: 34/200, Train Loss: 0.12095
Epoch: 35/200, Train Loss: 0.12279
Epoch: 36/200, Train Loss: 0.12040
Epoch: 37/200, Train Loss: 0.12139
Epoch: 38/200, Train Loss: 0.11647
Epoch: 39/200, Train Loss: 0.11453
Epoch: 40/200, Train Loss: 0.12076
Epoch: 41/200, Train Loss: 0.11980
Epoch: 42/200, Train Loss: 0.12368
Epoch: 43/200, Train Loss: 0.12189
Epoch: 44/200, Train Loss: 0.12340
Epoch: 45/200, Train Loss: 0.11953
Epoch: 46/200, Train Loss: 0.11911
Epoch: 47/200, Train Loss: 0.11425
Epoch: 48/200, Train Loss: 0.11721
Epoch: 49/200, Train Loss: 0.11824
Epoch: 50/200, Train Loss: 0.11958
Epoch: 51/200, Train Loss: 0.11953
Epoch: 52/200, Train Loss: 0.11598
Epoch: 53/200, Train Loss: 0.11955
Epoch: 54/200, Train Loss: 0.12507
Epoch: 55/200, Train Loss: 0.12238
Epoch: 56/200, Train Loss: 0.11752
Epoch: 57/200, Train Loss: 0.11934
Epoch: 58/200, Train Loss: 0.11551
Epoch: 59/200, Train